<a href="https://colab.research.google.com/github/PrannoyNamala/SuttonBartoExercises/blob/main/Chapter_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Exercise 2.1:

In e-greedy action selection, for the case of two actions and ϵ = 0.5, what is the probability that the greedy action is selected?

with epislon 0.5, the probability of selecting greedy action in (1-ϵ) + (1/n) * ϵ = 0.5 + (1/2) * 0.5 = 0.75

### Exercise 2.2:

Consider a k-armed bandit problem with k = 4 actions, denoted 1, 2, 3, and 4. Consider applying to this problem a bandit algorithm using e-greedy action selection, sample-average action-value estimates, and initial estimates of Q_1(a) = 0, for all a. Suppose the initial sequence of actions and rewards is A_1 = 1, R_1 = -1, A_2 = 2, R_2 = 1, A_3 = 2, R_3 = -2, A_4 = 2, R_4 = 2, A_5 = 3, R_5 = 0. On some of these time steps the ϵ case may have occurred, causing an action to be selected at random. On which time steps did this definitely occur? On which time steps could this possibly have occurred?

t=0: returns [-1,0,0,0]

A_1=1 returns R_1=-1  

t=1: returns [-1,0,0,0]; making 2,3,4 the greedy actions for next step.

A_2=2 returns R_2= 1

t=2: returns [-1,1,0,0]; making 2 the greedy action for next step.

A_3=2 returns R_3=-2

t=3: returns [-1,(1-2)/2,0,0]; making 3,4 the greedy action for next step.

A_4=2 returns R_4=2 ;  

t=4: returns [-1,(1-2+2)/3,0,0]; making 2 the greedy action for next step.

A_5=3 returns R_5=0

t=5: returns [-1,(1-2+2)/3,0,0]; making 2 the greedy action for next step

A_4 and A_5 are the timecases where it definitely occured. A_1, A_2, A_3 is a could be cause

### Exercise 2.3

In the comparison shown in Figure 2.2, which method will perform best in the long run in terms of cumulative reward and probability of selecting the best action? How much better will it be? Express your answer quantitatively.

The answer is eps=0.01

In the longrun, when all the true values of actions are explored, the eps-0.01 method will select the best action with a probability of 0.99 where as eps-0.1 will select the best action with a probability of 0.9

### Exercise 2.4 [TO-DO]

If the step-size parameters, 𝝰_n, are not constant, then the estimate Q_n is a weighted average of previously received rewards with a weighting different from that given by (2.6). What is the weighting on each prior reward for the general case, analogous to (2.6), in terms of the sequence of step-size parameters?

### Exercise 2.5 (programming)

Design and conduct an experiment to demonstrate the difficulties that sample-average methods have for non-stationary problems. Use a modified version of the 10-armed testbed in which all the q*(a) start out equal and then take independent random walks (say by adding a normally distributed increment with mean zero and standard deviation 0.01 to all the q*(a) on each step). Prepare plots like Figure 2.2 for an action-value method using sample averages, incrementally computed, and another action-value method using a constant step-size parameter, α = 0.1. Use ϵ = 0.1 and longer runs, say of 10,000 steps.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Parameters ##
k = 10
steps = 10000
runs = 2000
random_walk_mean = 0
random_walk_var = 0.01
eps = 0.1
alpha = 0.1
methods = ["incremental_computation", "constant_alpha"]   # "sample_averages",
################

total_rewards = {method: np.zeros(steps) for method in methods}

for run in range(runs):
  # Create 10 distributions with mean 0 and varience 1
  reward_mean_list = np.array([0 for _ in range(k)], dtype=np.float64)

  # Initialize action value estimates and action counts
  action_value_estimates = {
                            "sample_averages":np.array([0 for _ in range(k)]),
                            "incremental_computation":np.array([0 for _ in range(k)]),
                            "constant_alpha":np.array([0 for _ in range(k)])
                            }

  action_counts = {
                  "sample_averages":[[0]  for _ in range(k)], # Storing reward values here
                  "incremental_computation":np.array([0 for _ in range(k)]),
                  "constant_alpha":np.array([0 for _ in range(k)])
                  }

  rew_obtained = {
                  "sample_averages":[],
                  "incremental_computation":[],
                  "constant_alpha":[]
                  }


  # Looping over number of steps
  for step in range(steps):
    # Going through each method
    for estimate_type in action_value_estimates.keys():
      if estimate_type == "sample_averages":
        # Select action with eps-greedy
        action = np.argmax(action_value_estimates[estimate_type]) if np.random.random() > eps else np.random.randint(k)
        # Obtain reward for the action
        reward = np.random.normal(reward_mean_list[action], 1)

        rew_obtained[estimate_type].append(reward)

        # Store action reward
        action_counts[estimate_type][action].append(reward)

        # update estimates
        action_value_estimates[estimate_type][action] = np.mean(action_counts[estimate_type][action])
      else:
        # Select action with eps-greedy
        action = np.argmax(action_value_estimates[estimate_type]) if np.random.random() > eps else np.random.randint(k)
        # Obtain reward for the action
        reward = np.random.normal(reward_mean_list[action], 1)

        rew_obtained[estimate_type].append(reward)

        # update counts
        action_counts[estimate_type][action] += 1
        # Choosing step_size 1/n if incremental_computation and alpha if constant_alpha
        step_size = alpha if estimate_type == "constant_alpha" else 1/action_counts[estimate_type][action]
        # update estimates
        action_value_estimates[estimate_type][action] += step_size * (reward - action_value_estimates[estimate_type][action])


    # Random walk means addition
    reward_mean_list += np.random.normal(random_walk_mean, random_walk_var, size=k)

  for method in methods:
    total_rewards[method] += np.array(rew_obtained[method])

avg_rewards = {method: total_rewards[method] / runs for method in methods}

## Ploting
plt.figure(figsize=(12, 8))
for method in methods:
    plt.plot(avg_rewards[method], label=method)

plt.xlabel("Steps")
plt.ylabel("Average Reward")
plt.title("Average Reward over Steps for Different Action-Value Methods (Averaged over Runs)")
plt.legend()
plt.grid(True)
plt.show()

### Exercise 2.6 (Mysterious Spikes)

The results shown in Figure 2.3 should be quite reliable because they are averages over 2000 individual, randomly chosen 10-armed bandit tasks.
Why, then, are there oscillations and spikes in the early part of the curve for the optimistic method? In other words, what might make this method perform particularly better or worse, on average, on particular early steps?

When the agent explores the environment initially, all the initial values are vastly higher than what they actually are. Since the agent is greedy (eps=0), it only explores when the obtained reward is less than the estimated value. Luckily, sometimes these optimistic values overlap with the optimal action for the given step. After the selection, its value estimate is updated and the selection frequency drops. This can cause the agent to select suboptimal actions initially, causing dips in performance. As the agents select these actions multiple times, we get a much more accurate estimate from wich the optimal action% increases

### Exercise 2.7 (Unbiased Constant-Step-Size Trick)

In most of this chapter we have used sample averages to estimate action values because sample averages do not produce the initial bias that constant step sizes do (see the analysis in (2.6)). However, sample averages are not a completely satisfactory solution because they may perform poorly on non-stationary problems. Is it possible to avoid the bias of constant step sizes while retaining their advantages on non-stationary problems? One way is to use a step size of

β_n = α/ō_n

where α > 0 is a conventional constant step size and of is a trace of one that starts at 0:

ō_{n+1} = ō_{n-1} + α(1-ō_{n-1}) for n≥0, with ō=0

Carry out an analysis like that in (2.6) to show that Be is an exponential recency-weighted average without initial bias.

### Exercise 2.8 (UCB Spikes)

In Figure 2.4 the UCB algorithm shows a distinct spike in performance on the 11th step. Why is this? Note that for your answer to be fully satisfactory it must explain both why the reward increases
on the 11th step and why it decreases on the subsequent steps. Hint: if c = 1, then the spike is less
prominent.

### Exercise 2.9

Show that in the case of two actions, the soft-max distribution is the same as that given by the logistic, or sigmoid, function often used in statistics and artificial neural networks.

### Exercise 2.10

Suppose you face a 2-armed bandit task whose true action values change randomly from time step
to time step. Specifically, suppose that, for any time step, the true values of actions 1 and 2 are respectively 0.1 and 0.2 with probability 0.5 (case A), and 0.9 and 0.8 with probability 0.5 (case B). If you are not able to tell which case you face at any step, what is the best expectation of success you can achieve and how should you behave to achieve it? Now suppose that on each step you are told whether you are facing case A or case B (although you still don't know the true action values). This is an associative search task. What is the best expectation of success you can achieve in this task, and how should you behave to achieve it?

### Exercise 2.11 (programming)

Make a figure analogous to Figure 2.6 for the non-stationary case outlined in Exercise 2.5. Include the constant-step-size ϵ-greedy algorithm with a = 0.1. Use runs of 200,000 steps and, as a performance measure for each algorithm and parameter setting, use the average reward over the last 100,000 steps.